### In Scikit-learn
In this unit, we will see how to implement linear regressions with the Scikit-learn library.

#### Linear regression
First, let’s start by creating the X and y Numpy arrays.

In [1]:
import pandas as pd

# Load data
data_df = pd.read_csv("c3_marketing-campaign.csv")
X = data_df.drop("sales", axis=1).values
y = data_df.sales.values

The Scikit-learn library provides a LinearRegression object that uses the OLS method to fit a linear regression model. Just like the HuberRegressor and the SGDRegressor ones, this object implements the fit() function. In the Scikit-learn jargon, these objects are estimators and the function is part of the estimator API.

In [2]:
from sklearn.linear_model import LinearRegression

# Create a linear regression object
lr = LinearRegression()

# Fit the model
lr.fit(X, y)

# Print coefficients
print("Coefficients:", lr.coef_)
print("Intercept:", lr.intercept_)

Coefficients: [0.39465146 0.47037002 0.30669954]
Intercept: 0.024870917888196065


The LinearRegression() object will automatically compute the intercept term w0. We can access the parameters with the coef_ and the intercept_ attributes.

#### Predict score functions
Scikit-learn estimators also implement a predict() and a score() function.

The predict(X) function uses the coef_ and the intercept_ attributes to compute predictions for the data points in the X matrix.



In [3]:
# Compute predictions
y_pred = lr.predict(X)
y_pred[:3]

array([1.24462012, 4.84934038, 4.04266482])

We can also manually compute these predictions with the matmul() function from Numpy.

In [4]:
import numpy as np

y_pred = np.matmul(X, lr.coef_) + lr.intercept_
y_pred[:3]

array([1.24462012, 4.84934038, 4.04266482])

The score(X, y) function computes the predictions for the data points in X and evaluates the R2 coefficient using the target values in y.

In [5]:
# Compute the R2 coefficient
R2 = lr.score(X, y)
print("R2:", R2)

R2: 0.9832893048848236


#### SGDRegressor

The OLS method is the most common way to find the parameters of a linear regression model that minimize the squares of the residuals. However, in the unit about Huber loss, we saw the SGDRegressor object which implements a variant of the gradient descent (GD) algorithm.

It’s important to understand that this algorithm doesn’t compute an analytical solution. It’s an iterative algorithm that tries to get closer to the optimal solution after each iteration. However, unlike the OLS method, gradient descent is very generic and can optimize many different cost functions, e.g., Huber loss.

To minimize the squares of the residuals, we can set its loss parameter to squared_error. By default, it adds a penalization term to the cost function. We will learn more about it later in this course. As for now, we can set it 'none'.

In [6]:
from sklearn.linear_model import SGDRegressor

# Create the SGDRegressor object
lr_sgd = SGDRegressor(
    loss="squared_error",  # Cost function
    penalty="none",  # Add a penalty term?
    max_iter=1000,  # Number of iterations
    random_state=0,  # The implementation shuffles the data
    tol=1e-3,  # Tolerance for improvement (stop SGD once loss is below)
)

# Fit the linear regression model
lr_sgd.fit(X, y)

# Print coefficients
print("Coefficients:", lr_sgd.coef_)
print("Intercept:", lr_sgd.intercept_)

Coefficients: [0.39968853 0.44409771 0.25894341]
Intercept: [0.12807209]


The implementation of the SGDRegressor object shuffles the data before running the optimization algorithm. To get the results from above, you should set its random_state parameter to zero.

The object also implements the estimator API. For instance, we can compute the R2 with the score() function.

In [7]:
# Compute R2 coefficient
R2_sgd = lr_sgd.score(X, y)
print("R2_sgd:", R2_sgd)

R2_sgd: 0.9821546772612869


#### Huber loss
We usually fit linear regressions using the least squares approach, i.e., minimizing the squares of the residuals. However, it’s also possible to use other objective functions such as Huber loss.

To achieve this, we can create an HuberRegressor object which also implements the estimator interface.

In [8]:
from sklearn.linear_model import HuberRegressor

# Create the estimator
huber = HuberRegressor(epsilon=1.35)

# Fit it to X,y
huber.fit(X, y)

print("Coefficients:", huber.coef_)
print("Intercept:", huber.intercept_)
print("R^2 coefficient:", huber.score(X, y))


Coefficients: [0.39172544 0.4788203  0.29315421]
Intercept: 0.04586298819194033
R^2 coefficient: 0.9830701571142849


### Ill-conditioning

In this unit, we will learn about ill-conditioning which can cause numerical issues when computing the ordinary least squares solution (OLS). We will start by describing collinearity which is a phenomenon closely related to ill-conditioning. Then, we will see how nearly collinear features can cause numerical issues. Finally, we will experiment with regularization which is a way to handle ill-conditioning.

#### Collinearity
Collinearity (and multicollinearity) happens when there is an exact linear relationship between one or more features in the input matrix 
X. This means that one feature is a combination of the others plus some constant. This may sound abstract, so let’s take an example.

In [10]:
import pandas as pd
import numpy as np

# Load data
data_df = pd.read_csv("c3_bike-sharing.csv")

# Create Numpy arrays
temp = data_df.temp.values
users = data_df.users.values

# Create collinear feature
temp_C = 47 * temp - 8


# Create input matrix X
X = np.c_[temp, temp_C]

# Add a column of ones
X1 = np.c_[np.ones(X.shape[0]), X]

# Compute rank
rank = np.linalg.matrix_rank(X1)
print("Rank", rank)

Rank 2


#### Collinearity in practice
In theory, the OLS solution doesn’t exist when X contains collinear features. However, most machine learning tools can handle these situations and return a vector of parameters w as if there were no collinear features.

In [11]:
from scipy.linalg import lstsq

# Compute OLS using lstsq
w, rss, rank, _ = lstsq(X1, users)

print("w:", w)
print("rank:", rank)
print("RSS:", rss)

w: [155.34445517  27.10638524  31.24446504]
rank: 2
RSS: []


In this code, we call the lstsq() function with the rank-deficient matrix X1 which returns a set of parameters w, the rank of the matrix, and the RSS score which is an array of length zero when the matrix is rank-deficient. The function also returns the singular values of the matrix (the fourth return value), but we can discard this result by assigning it to the “throwaway” variable _.

Let’s compare the performance of this model to a simple linear regression with the R2 coefficient. We can use the r2_score(y,y_pred) function from the Scikit-learn metrics module.

In [12]:
from sklearn.metrics import r2_score

# R^2 coefficient of simple linear regression
coefs = np.polyfit(temp, users, deg=1)
y_pred_normal = np.polyval(coefs, temp)
r2_normal = r2_score(users, y_pred_normal)
print("R^2 normal:", r2_normal)

R^2 normal: 0.5954233080185317


In [13]:
# R^2 coefficient with collinear features
y_pred_collinear = np.matmul(X1, w)
r2_collinear = r2_score(users, y_pred_collinear)
print("R^2 collinear:", r2_collinear)

R^2 collinear: 0.5954233080185317


#### Nearly collinear features
Sometimes, features are highly correlated but there isn’t a perfect linear relationship between them. These are nearly collinear features.

For instance, say that we measure temperatures with two different thermometers. One gives temperatures in degrees Celsius and the other in degrees Fahrenheit. To simulate this scenario, we can simply convert the temp_C values to degrees Fahrenheit.

In [28]:
# Convert to degrees Celsius to Fahrenheit
temp_F = 1.8 * temp_C + 32

It’s unlikely that the two thermometers give the same temperature values. In that sense, temp_F and temp_C are nearly collinear features. To simulate this difference, we can add a small noise to the temp_F variable.

In [29]:
def compute_ols_with_noise(temp_C, users):

    # Convert to degrees Fahrenheit
    temp_F = 1.8 * temp_C + 32

    # Add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_F.shape)
    temp_F += noise

    # Create input matrix X
    X = np.c_[temp_C, temp_F]

    # Compute OLS using lstsq
    X1 = np.c_[np.ones(X.shape[0]), X]  # Create X1 matrix
    w, rss, rank, _ = lstsq(X1, users)  # OLS

    return w, rss, rank


w, rss, rank = compute_ols_with_noise(temp_C, users)

print("rank:", rank)  # Returns: 3
print("RMSE:", np.sqrt(rss / len(users)))  # Depends on the noise value
print("w:", w)  # Depends on the noise value


rank: 3
RMSE: 233.36492111651108
w: [-2854.42227205  -137.72401678    94.19559382]


In this code, we generate a random noise using a Gaussian distribution centered at zero and with a standard deviation of 0.01.

This time, X1 is full rank and the lstsq() function returns the residual sum of squares. If you run the code several times, you can see that the coefficients vary a lot.


In [30]:
txt_fmt = "{:<5}{:<6}{:<20}{:}"
print(txt_fmt.format("run", "rank", "RMSE", "coefficients"))
for i in range(5):
    w, rss, rank = compute_ols_with_noise(temp_C, users)  # Compute OLS using lstsq
    print(txt_fmt.format(i, rank, np.sqrt(rss / len(users)), w))

run  rank  RMSE                coefficients
0    3     233.35265611649376  [8380.88005102  494.22239743 -256.8946287 ]
1    3     230.65134740503473  [115372.43240063   6513.04443211  -3600.58950406]
2    3     232.9053992808012   [-47757.71083779  -2663.51856806   1497.39644333]
3    3     233.35122334582982  [-8349.62033404  -446.8559625    265.92552603]
4    3     232.9859176013226   [39252.97238344  2230.65105489 -1221.608021  ]


This is due to ill-conditioning.
#### Ill-conditioning

In the example from above, X1 has full-rank and the OLS solution exists. However, it’s numerically unstable. A small change in the data produces very different coefficients. We can quantify this phenomenon with the condition number. Inverting a matrix with a large condition number is numerically unstable.

We can compute the condition number of X1 with the cond() function from the Numpy linalg module.

In [31]:
# Condition number
cn = np.linalg.cond(X1)
print("Condition number:", cn)  # Depends on the noise value

Condition number: 8.720781054454994e+16


The value of cn depends on the noise in the X1 matrix from above. However, you should get a value above 200 thousand. By increasing the scale of the noise, you decrease the correlation between the two variables and you should see that it reduces the condition number.

Again, ill-conditioning doesn’t necessarily affect the predictive accuracy of the model. In most cases, it will simply result in large variations in the model coefficients.



In [32]:
# Same with the nearly collinear matrix
y_pred_nearcol = np.matmul(X1, w)
r2_nearcol = r2_score(users, y_pred_nearcol)

# R^2 coefficient with nearly collinear features
print("R^2 nearly collinear:", r2_nearcol)

R^2 nearly collinear: -3701.1660566799164


#### Regularization
One way to solve ill-conditioning is to create a constraint on the coefficients. The idea is to modify the objective function and add a regularization term that penalizes large coefficients.

Scikit-learn implements regularization with the Ridge estimator which is similar to the LinearRegression one.

In [26]:
from sklearn.linear_model import Ridge


def compute_with_regularization(temp_C, users):

    # Add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_C.shape)
    temp_F = (1.8 * temp_C + 32) + noise

    # Create input matrix X
    X = np.c_[temp_C, temp_F]

    # Fit a Ridge regression
    ridge = Ridge(alpha=100)
    ridge.fit(X, users)

    return ridge

ridge = compute_with_regularization(temp_C, users)

print("Coefficients:", ridge.coef_)
print("Intercept:", ridge.intercept_)
print("R^2:", ridge.score(X, users))

Coefficients: [ 7.41318097 13.5428751 ]
Intercept: -272.8970368939782
R^2: -3.5627161779952363


In [27]:
txt_fmt = "{:<5}{:<27}{:<21}{:}"
print(txt_fmt.format("run", "coefficients", "intercept", "R^2"))
for i in range(5):
    ridge = compute_with_regularization(temp_C, users)
    print(txt_fmt.format(i, str(list(ridge.coef_.round(8))),
                         ridge.intercept_, ridge.score(X, users)))

run  coefficients               intercept            R^2
0    [7.33642015, 13.58584582]  -274.284550888916    -3.569639532624815
1    [7.58836506, 13.44538498]  -269.78181632857707  -3.547392655357105
2    [7.86625335, 13.29049328]  -264.80637006647     -3.522868071159177
3    [7.6399375, 13.41573798]   -268.7982151340657   -3.542360283661595
4    [7.22076525, 13.65002799]  -276.32792079865294  -3.5796884320989157


In this code, we create an input matrix X of nearly collinear features, and we fit the Ridge model to it. Note that we pass the X matrix to the fit() function (i.e., without the column of ones) since Scikit-learn objects automatically compute the intercept term. Also, we use an alpha parameter which controls the regularization strength. Finally, we print the model coefficients and the R2 score.
If you run the code several times, you should see that the coefficient are more stable than before.

Let’s summarize what we’ve learned in this unit. Here are a few takeaways.

* Collinearity happens when there is an exact linear relationship between one or more features. In this case, the X matrix, with the additional column of ones, and its moment matrix XtX are rank-deficient and the OLS solution doesn’t exist.
* Nearly collinear features make computations numerically unstable which result in large variations in the model coefficients. This is called ill-conditioning. It’s numerically unstable to compute the inverse of matrices with a large condition number.
* Regularization stabilizes model coefficients.
Ill-conditioning is a complex topic. You can take a look at this thread if you want to learn more about it. However, note that some answers refer to models that we will see later in this course.

In the next exercise, you will fit a linear regression model to a dataset with multiple features (including collinear and nearly collinear ones). This is an excellent opportunity to experiment with ill-conditioning.